In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("udf").getOrCreate()

transactions = [
    ('에그드랍', '2022-11-20 15:20:00', 15700, 'KRW'),
    ('두마리찜닭', '2022-11-18 19:50:00', 32800, 'KRW'), 
    ('커피온리', '2022-11-19 12:25:00', 15900, 'KRW'), 
    ('뉴발란스 후리스', '2022-11-20 13:20:00', 45000, 'KRW'), 
    ('아로마티카 시카', '2022-11-20 00:01:30', 18800, 'KRW')
]

schema = ["name", "datetime", "price", "currency"]

In [3]:
df=spark.createDataFrame(data= transactions, schema=schema)
df.createOrReplaceTempView("transactions")

In [4]:
spark.sql("select * from transactions").show()

+---------------+-------------------+-----+--------+
|           name|           datetime|price|currency|
+---------------+-------------------+-----+--------+
|       에그드랍|2022-11-20 15:20:00|15700|     KRW|
|     두마리찜닭|2022-11-18 19:50:00|32800|     KRW|
|       커피온리|2022-11-19 12:25:00|15900|     KRW|
|뉴발란스 후리스|2022-11-20 13:20:00|45000|     KRW|
|아로마티카 시카|2022-11-20 00:01:30|18800|     KRW|
+---------------+-------------------+-----+--------+



# UDF
- user defined function : 사용자 정의 함수
- 분산 병렬 처리 환경에서 사용할 수 있는 함수를 만들어 낸다.
    - spark에서 기본적으로 제공하지 않는 함수를 worker에서 일을 할 수 있도록 함수를 만들어준다.
- udf를 만들지 않고 하면 마스터노드에서 일을 하는 상태이다.
    - ``워커한테 일을 할 수 있는 함수``를 만들어준다.

In [5]:
from pyspark.sql.types import LongType
# 마스터 노드에서 사용하는 함수이다. worker에서 작동하지 않는다.
def squared(n):
    return n*n

#### worker에서 함수가 작동되게 할 수 있도록 udf로 등록

In [6]:
spark.udf.register("squared",squared,LongType())
# 가로 안 첫번째 문자열은 worker에서 사용할 함수의 이름 지정
# 가로 안 두번째는 내가 작성한 함수이름을 입력
# 가로 안 세번째 리턴타입(지정하지 않으면 디폴트 string 문자열)

<function __main__.squared(n)>

In [7]:
spark.sql("SELECT price, squared(price) FROM transactions").printSchema()
# squared(price)는  spark.udf.register("squared",squared,LongType())의 "squared"에서 온다.

root
 |-- price: long (nullable = true)
 |-- squared(price): long (nullable = true)



In [8]:
spark.sql("SELECT price, squared(price) as udf_amount FROM transactions").show()

+-----+----------+
|price|udf_amount|
+-----+----------+
|15700| 246490000|
|32800|1075840000|
|15900| 252810000|
|45000|2025000000|
|18800| 353440000|
+-----+----------+



#### 숫자 한글로 변경하는 함수

In [9]:
def read_number(n):
    units = ["", "십", "백", "천", "만"]
    nums = '일이삼사오육칠팔구'
    result = []
    i = 0
    while n > 0:
        n, r = divmod(n, 10)
        if r > 0:
            result.append(nums[r-1]+units[i])
        i += 1
    return "".join(reversed(result))

In [10]:
read_number(35000)

'삼만오천'

In [11]:
spark.udf.register("read_number",read_number) # 리턴 타입이 생략된 것은 문자열 타입으로 리턴이 된다.

<function __main__.read_number(n)>

In [12]:
query = '''
select price, read_number(price)
from transactions
'''

spark.sql(query).show()

+-----+------------------+
|price|read_number(price)|
+-----+------------------+
|15700|      일만오천칠백|
|32800|      삼만이천팔백|
|15900|      일만오천구백|
|45000|          사만오천|
|18800|      일만팔천팔백|
+-----+------------------+



#### 요일 구해내는 함수 만들고 등록하기

In [13]:
# 요일 구해내는 함수 만들기
def get_weekday(date):
    import calendar
    return calendar.day_name[date.weekday()]

spark.udf.register("get_weekday", get_weekday)

<function __main__.get_weekday(date)>

In [14]:
from datetime import datetime
get_weekday(datetime(2022,11,21) )

'Monday'

In [15]:
query = """

SELECT datetime, get_weekday(TO_DATE(datetime)) as day_of_week
FROM transactions
"""

spark.sql(query).show()

+-------------------+-----------+
|           datetime|day_of_week|
+-------------------+-----------+
|2022-11-20 15:20:00|     Sunday|
|2022-11-18 19:50:00|     Friday|
|2022-11-19 12:25:00|   Saturday|
|2022-11-20 13:20:00|     Sunday|
|2022-11-20 00:01:30|     Sunday|
+-------------------+-----------+



In [16]:
spark.stop()